In [1]:
# Compile c++ physics engine
!c++ -O3 -Wall -shared -std=c++11 -fPIC $(python3 -m pybind11 --includes) SPHEngine_Pybind.cpp -o fluidSim$(python3-config --extension-suffix)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from math import pi
import fluidSim  # The pybind11 module compiled from fluid_sim.cpp

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as colors

%matplotlib qt


In [3]:
# Simulation constants (could also be taken from fluidSim if desired)
SIM_W = fluidSim.SIM_W
SIM_H = fluidSim.SIM_H
BOTTOM = fluidSim.BOTTOM
TOP = fluidSim.TOP
G_ANG = fluidSim.G_ANG
G_MAG = fluidSim.G_MAG
N = 350  # Number of particles
VAR_INTENSITY = True
MAX_INTENSITY = 1

# Create the simulation instance with particles randomly placed in the domain.
sim = fluidSim.Simulation(N, -SIM_W, SIM_W, BOTTOM, TOP)
frame = 0
frame_grid = 0

In [4]:
# ------------- Particle Animation -----------------
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, xlim=(-SIM_W, SIM_W), ylim=(0, SIM_H))
points, = ax.plot([], [], "o", ms=20, color="black")

def animate(i):
    global frame
    # Update simulation state (varying g_ang as an example)
    sim.update(G_MAG, G_ANG + frame * pi / 100)
    # Get particle positions as a flat list and reshape to (N,2)
    pos = np.array(sim.get_visual_positions()).reshape(-1, 2)
    points.set_data(pos[:, 0], pos[:, 1])
    frame += 1
    return (points,)

ani = animation.FuncAnimation(fig, animate, interval=10, blit=True)
plt.show()

/tmp/ipykernel_7520/1200804822.py:16: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, animate, interval=10, blit=True)


In [5]:
# ------------- Grid Animation -----------------
if VAR_INTENSITY:
    custom_cmap = LinearSegmentedColormap.from_list('white_to_black', ['white', 'black'])
    max_intensity = 4
    norm = colors.Normalize(vmin=0, vmax=max_intensity)


def hash_grid(positions, sim_w, sim_h, cell_size=0.1, binary=True):
    # Compute grid dimensions
    nx = int((2 * sim_w) / cell_size)
    ny = int(sim_h / cell_size)
    grid = np.zeros((ny, nx), dtype=int)
    for (x, y) in positions:
        x_index = int((x + sim_w) / cell_size)
        y_index = int(y / cell_size)
        if 0 <= x_index < nx and 0 <= y_index < ny:
            if binary:
                grid[y_index, x_index] = 1
            else:
                grid[y_index, x_index] += 1
    return grid

fig_grid = plt.figure()
ax_grid = fig_grid.add_subplot(1, 1, 1, xlim=(-SIM_W, SIM_W), ylim=(0, SIM_H))
# Get an initial grid image
init_pos = np.array(sim.get_visual_positions()).reshape(-1, 2)
init_grid = hash_grid(init_pos, SIM_W, SIM_H, cell_size=0.1, binary=not VAR_INTENSITY)
GRID_IM = ax_grid.imshow(init_grid, cmap="binary" if not VAR_INTENSITY else custom_cmap, origin='lower',
                         extent=(-SIM_W, SIM_W, 0, SIM_H), interpolation='none')

def animate_grid(i):
    global frame_grid
    sim.update(G_MAG, G_ANG + frame_grid * np.pi/75)
    pos = np.array(sim.get_visual_positions()).reshape(-1, 2)
    grid_data = hash_grid(pos, SIM_W, SIM_H, cell_size=0.1, binary=not VAR_INTENSITY)
    GRID_IM.set_data(grid_data)
    frame_grid += 1
    return (GRID_IM,)

ani_grid = animation.FuncAnimation(fig_grid, animate_grid, interval=10, blit=True)

/tmp/ipykernel_7520/1439353591.py:40: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani_grid = animation.FuncAnimation(fig_grid, animate_grid, interval=10, blit=True)
